# 7章 畳み込みニューラルネットワーク

In [26]:
import sys, os
sys.path.append(os.pardir)
sys.path.append(f"{os.pardir}/deep-learning-from-scratch")

import numpy as np
import pickle
from beartype import beartype
from collections import OrderedDict
from nptyping import *
from typing import Any
from common.layers import SoftmaxWithLoss
from common.util import im2col,col2im

In [27]:
x = np.random.rand(10,1,28,28)
x.shape
# (10, 1, 28, 28) ミニバッチ, チャンネル, 高さ, 幅

(10, 1, 28, 28)

In [28]:
x1 = np.random.rand(1,3,7,7)
col1 = im2col(x1, 5, 5, stride=1, pad=0)
print(col1.shape)   # (9, 75) フィルターの適用領域の数(=1*(((7-5)/1)+1)**2), 入力特徴マップの要素数

(9, 75)


In [29]:
print(x1[0][0])
print(col1[0])

[[0.03009721 0.9060595  0.18353027 0.69039328 0.79496692 0.42859478
  0.61670357]
 [0.68096794 0.67593603 0.45722314 0.44227229 0.17771988 0.36160591
  0.20488367]
 [0.45136367 0.3245889  0.62791324 0.44165241 0.04922141 0.95710857
  0.47995189]
 [0.57162995 0.63351797 0.35959455 0.66194534 0.17462996 0.5563603
  0.48300417]
 [0.23532974 0.46221763 0.34139294 0.60160882 0.24554715 0.20592776
  0.26682934]
 [0.73867696 0.18110712 0.53638357 0.90372435 0.59366502 0.92501818
  0.25098507]
 [0.44178977 0.63625207 0.54189992 0.40167296 0.20283005 0.69205657
  0.60465544]]
[0.03009721 0.9060595  0.18353027 0.69039328 0.79496692 0.68096794
 0.67593603 0.45722314 0.44227229 0.17771988 0.45136367 0.3245889
 0.62791324 0.44165241 0.04922141 0.57162995 0.63351797 0.35959455
 0.66194534 0.17462996 0.23532974 0.46221763 0.34139294 0.60160882
 0.24554715 0.71370169 0.61150628 0.32000952 0.13695608 0.29002464
 0.21893195 0.38466954 0.48108638 0.43649887 0.16879082 0.12972231
 0.65984619 0.29924884 0.

In [30]:
x2 = np.random.rand(10,3,7,7)
col2 = im2col(x2, 5, 5, stride=1, pad=0)
print(col2.shape)   # (90, 75) フィルターの適用領域の数(=10*(((7-5)/1)+1)**2), 入力特徴マップの要素数

(90, 75)


In [31]:
def output_size(x_len, pad, filter_len, stride) -> int:
    rest = ((x_len + 2 * pad) - filter_len)
    if rest < 0:
        raise Exception(f"Filter length {filter_len} is longer than input size {x_len} + {pad}!")
    elif rest % stride != 0:
        raise Exception(f"Rest length {rest} and stride {stride} are conflicted!")
    else:
        return int(rest / stride + 1)  # reshapeにも使うのでintにする

class Convolution:
    # 今回はフィルターのチャンネル数を3で固定している
    @beartype
    def __init__(self, W: NDArray[Shape['FN,C,FH,FW'],Float], b: NDArray[Shape['FN'], Float], stride=1, pad=0):
        # FN: Filter Number
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad

        # 中間データ（backward時に使用）
        self.x = None   
        self.col = None
        self.col_W = None
        
        # 重み・バイアスパラメータの勾配
        self.dW = None
        self.db = None

    @beartype
    def forward(self, x: NDArray[Shape['N,C,H,W'],Any]):
        # Float32が送られてくる。コード全体を通してFloat64の場合もあり使い分けがよく分からないので、いったん型は指定しない。
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = output_size(H, self.pad, FH, self.stride)
        out_w = output_size(W, self.pad, FW, self.stride)

        col: NDArray[Shape['N*Out_h*Out_w,C*FH*FW'], Float] = im2col(x, FH, FW, self.stride, self.pad)
        col_W: NDArray[Shape['C*FH*FW,FN'], Float] = self.W.reshape(FN, -1).T
        out: NDArray[Shape['N*Out_h*Out_w,FN', Float]] = np.dot(col, col_W) + self.b
        reshaped_out: NDArray[Shape['N,FN,Out_h,Out_w'], Float] = out.reshape(N, out_h, out_w, FN).transpose(0,3,1,2)

        self.x = x
        self.col = col
        self.col_W = col_W

        return reshaped_out

    @beartype
    def backward(self, dout: NDArray[Shape['N,FN,Out_h,Out_w'], Float]):
        FN, C, FH, FW = self.W.shape
        dout_matrix: NDArray[Shape['N*Out_h*Out_w,FN'], Float] = dout.transpose(0,2,3,1).reshape(-1, FN)

        self.db: NDArray[Shape['1,N'], Float] = np.sum(dout_matrix, axis=0)
        dW_matrix: NDArray[Shape['C*FH*FW,FN'], Float] = np.dot(self.col.T, dout_matrix)
        self.dW: NDArray[Shape['FN,C,FH,FW']] = dW_matrix.transpose(1, 0).reshape(FN, C, FH, FW)

        dcol: NDArray[Shape['N*Out_h*Out_w,C*FH*FN'], Float] = np.dot(dout_matrix, self.col_W.T)
        dx: NDArray[Shape['N,C,H,W']] = col2im(dcol, self.x.shape, FH, FW, self.stride, self.pad)

        return dx


In [32]:
class Pooling:
    def __init__(self, pool_h, pool_w, stride=2, pad=0):
        # pool_h, pool_wはそれぞれプーリング適用領域の高さ・幅。例えば3x3=9からmaxを取るなら、pool_h=3, pool_w=3
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad

        self.x = None
        self.arg_max: NDArray[Shape['N*Out_h*Out_w*C,1'], Int] = None

    @beartype
    def forward(self, x: NDArray[Shape['N,C,H,W'], Float]):
        # 出力特徴マップの奥行きを、対象が色ではないのにチャンネルと呼ぶのは個人的にまだ違和感があるが、そのうち慣れる。
        N, C, H, W = x.shape
        out_h = int(1 + (H - self.pool_h) / self.stride)
        out_w = int(1 + (W - self.pool_w) / self.stride)

        col: NDArray[Shape['N*Out_h*Out_w,C*Pool_h*Pool_w'], Float] = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        # im2colはフィルタ適用領域向けに実装されている。フィルタはチャンネルをまたいで適用されるが、プーリングはチャンネルごとに適用される。
        # そのため、colをチャンネルごとに改行することで、プーリング適用領域ごとに別々の行にする。
        reshaped_col: NDArray[Shape['N*Out_h*Out_w*C,Pool_h*Pool_w'], Float] = col.reshape(-1, self.pool_h*self.pool_w)
        arg_max = np.argmax(reshaped_col, axis=1)
        out: NDArray[Shape['N*C,1'], Float] = np.max(reshaped_col, axis=1)
        reshaped_out: NDArray[Shape['N,C,Out_h,Out_w'], Float] = out.reshape(N, out_h, out_w, C).transpose(0,3,1,2)

        self.x = x
        self.arg_max = arg_max

        return reshaped_out

    @beartype
    def backward(self, dout: NDArray[Shape['N,C,Out_h,Out_w'], Float]):
        dout = dout.transpose(0, 2, 3, 1)
        
        pool_size = self.pool_h * self.pool_w
        dmax = np.zeros((dout.size, pool_size))
        dmax[np.arange(self.arg_max.size), self.arg_max.flatten()] = dout.flatten()
        dmax = dmax.reshape(dout.shape + (pool_size,)) 
        
        dcol = dmax.reshape(dmax.shape[0] * dmax.shape[1] * dmax.shape[2], -1)
        dx = col2im(dcol, self.x.shape, self.pool_h, self.pool_w, self.stride, self.pad)
        
        return dx


In [33]:
# Rectifyは電流の交流を整流にすることから名付けられた。電流の交流は正負の電流が交互に流れるが、整流にすると正の電流のみが流れる。
class Relu:
    def __init__(self):
        self.mask = None

    @beartype
    def forward(self, x:NDArray):
        # NDArray[Shape['N,FN,Out_h,Out_w'], Any]|NDArray[Shape['N,Hidden_size'], Any] で定義したい。isinstance()では判定できるが、beartypeでは失敗するため諦めた。
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0

        return out

    @beartype
    def backward(self, dout: NDArray):
        dout[self.mask] = 0
        dx = dout

        return dx


In [34]:
class Affine:
    def __init__(self, W: NDArray[Shape['S,WS'], Float], b: NDArray[Shape['D'], Float]):
        self.W = W
        self.b = b
        self.x = None
        self.original_x_shape = None
        self.dW = None
        self.db = None

    @beartype
    def forward(self, x: NDArray):
        self.original_x_shape = x.shape
        x = x.reshape(x.shape[0], -1)
        self.x = x

        out = np.dot(x, self.W) + self.b

        return out

    @beartype
    def backward(self, dout: NDArray[Shape['N,WS'], Float]):
        dx: NDArray[Shape['N,S'], Float] = np.dot(dout, self.W.T)
        self.dW: NDArray[Shape['S,WS'], Float] = np.dot(self.x.T, dout)
        self.db: NDArray[Shape['1'], Float] = np.sum(dout, axis=0)

        dx = dx.reshape(*self.original_x_shape)  # 入力データの形状に戻す（テンソル対応）
        return dx

In [35]:
# 畳み込み層 → 全結合層(ReLU) → 全結合層(Softmax)を想定

class SimpleConvNet:
    @beartype
    def __init__(self, input_dim=(1,28,28),
        # filter_size:5は、5x5を表す。正方形がメジャー。
        conv_param={'filter_num':30, 'filter_size':5, 'pad':0, 'stride':1},
        hidden_size=100, output_size=10, weight_init_std=0.01):
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        input_size = input_dim[1]
        conv_output_size = int(1 + (input_size + 2*filter_pad - filter_size) / filter_stride)
        pool_output_size = int(filter_num * (conv_output_size/2) * (conv_output_size/2))

        self.params = {}
        self.params['W1']: NDArray[Shape['FN,C,FS,FS'],Float] = weight_init_std * np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
        self.params['b1'] = np.zeros(filter_num)
        self.params['W2']: NDArray[Shape['PS,HS']] = weight_init_std * np.random.randn(pool_output_size, hidden_size)
        self.params['b2'] = np.zeros(hidden_size)
        self.params['W3']: NDArray[Shape['HS,OS']] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b3'] = np.zeros(output_size)

        self.layers = OrderedDict()
        self.layers['Conv1'] = Convolution(self.params['W1'], self.params['b1'], conv_param['stride'], conv_param['pad'])
        self.layers['Relu1'] = Relu()
        self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
        self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
        self.layers['Relu2'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])

        self.last_layer = SoftmaxWithLoss()

    @beartype
    def predict(self, x: NDArray):
        for layer in self.layers.values():
            x = layer.forward(x)
        return x

    @beartype
    def loss(self, x: NDArray, t: NDArray[Shape['N'], Int]):
        y = self.predict(x)
        return self.last_layer.forward(y,t)

    @beartype
    def accuracy(self, x, t, batch_size=100):
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        acc = 0.0
        
        for i in range(int(x.shape[0] / batch_size)):
            tx = x[i*batch_size:(i+1)*batch_size]
            tt = t[i*batch_size:(i+1)*batch_size]
            y = self.predict(tx)
            y = np.argmax(y, axis=1)
            acc += np.sum(y == tt) 
        
        return acc / x.shape[0]

    @beartype
    def gradient(self, x: NDArray, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.last_layer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        grads = {}
        grads['W1'] = self.layers['Conv1'].dW
        grads['b1'] = self.layers['Conv1'].db
        grads['W2'] = self.layers['Affine1'].dW
        grads['b2'] = self.layers['Affine1'].db
        grads['W3'] = self.layers['Affine2'].dW
        grads['b3'] = self.layers['Affine2'].db

        return grads

    def save_params(self, file_name="params.pkl"):
        params = {}
        for key, val in self.params.items():
            params[key] = val
        with open(file_name, 'wb') as f:
            pickle.dump(params, f)


In [36]:
import numpy as np
import matplotlib.pyplot as plt
from dataset.mnist import load_mnist
from common.trainer import Trainer

# データの読み込み
(x_train, t_train), (x_test, t_test) = load_mnist(flatten=False)

# 処理に時間のかかる場合はデータを削減 
#x_train, t_train = x_train[:5000], t_train[:5000]
#x_test, t_test = x_test[:1000], t_test[:1000]

max_epochs = 20

network = SimpleConvNet(input_dim=(1,28,28), 
                        conv_param = {'filter_num': 30, 'filter_size': 5, 'pad': 0, 'stride': 1},
                        hidden_size=100, output_size=10, weight_init_std=0.01)
                        
trainer = Trainer(network, x_train, t_train, x_test, t_test,
                  epochs=max_epochs, mini_batch_size=100,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=1000)
trainer.train()

# パラメータの保存
network.save_params("params.pkl")
print("Saved Network Parameters!")

# グラフの描画
markers = {'train': 'o', 'test': 's'}
x = np.arange(max_epochs)
plt.plot(x, trainer.train_acc_list, marker='o', label='train', markevery=2)
plt.plot(x, trainer.test_acc_list, marker='s', label='test', markevery=2)
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.ylim(0, 1.0)
plt.legend(loc='lower right')
plt.show()

train loss:2.2992925771170176
=== epoch:1, train acc:0.136, test acc:0.161 ===
train loss:2.2961884053567148
train loss:2.2929277076926
train loss:2.282010918696065
train loss:2.2717066366264427
train loss:2.2534384450704854
train loss:2.244887587330623
train loss:2.213015578149984
train loss:2.19650226450476
train loss:2.169526605448429
train loss:2.1301282427389303
train loss:2.1406914246057585
train loss:2.035558550064247
train loss:1.9465087748485297
train loss:1.9159028778745864
train loss:1.9321146988155875
train loss:1.8189482293127608
train loss:1.687622170513817
train loss:1.6612734667376483
train loss:1.576816634444282
train loss:1.4871022354173684
train loss:1.3699508247554233
train loss:1.3904014098046094
train loss:1.255350354744139
train loss:1.0725085679035176
train loss:1.1127197184345865
train loss:1.063908932352259
train loss:0.9910471272562573
train loss:0.9082732054799044
train loss:0.8417009011250312
train loss:0.8241555182094876
train loss:0.7987650788221992
train

AttributeError: 'SimpleConvNet' object has no attribute 'save_params'